In [45]:
# Find out all tokens getting mapped to unk with Marian Tokenizer

In [46]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-af-fr")

/home/rana/Projects/zindi/transformers/src/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
from datasets import load_dataset,concatenate_datasets
zindi_ds = load_dataset("uvci/Koumankan_mt_dyu_fr")
def preprocess_function(examples):
    inputs = [example["dyu"] for example in examples["translation"]]
    targets = [example["fr"] for example in examples["translation"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=48, truncation=True, padding="max_length")
    # Check for None values in input_ids and labels
    if None in model_inputs["input_ids"] or None in model_inputs["labels"]:
        print("Warning: None values found in tokenized output")
        # Remove examples with None values
        valid_indices = [i for i, (inp, lab) in enumerate(zip(model_inputs["input_ids"], model_inputs["labels"]))
                         if inp is not None and lab is not None]
        for key in model_inputs.keys():
            model_inputs[key] = [model_inputs[key][i] for i in valid_indices]
    return model_inputs
tokenized_zds = zindi_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=zindi_ds["train"].column_names  # Remove original columns
)
concat_ds = concatenate_datasets([tokenized_zds['train'], tokenized_zds['test']])

Map: 100%|██████████| 1393/1393 [00:00<00:00, 2073.88 examples/s]


In [13]:
%cd ..

/home/rana/Projects/zindi


/home/rana/.cache/pypoetry/virtualenvs/zindi-z3yfXQo9-py3.9/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [18]:
with open('data/all_dyu.txt', 'r') as f:
    dyu_data = [line.strip() for line in f]
with open('data/all_fr.txt', 'r') as f:
    fr_data = [line.strip() for line in f]

all_data = dyu_data+fr_data
len(dyu_data), len(fr_data), len(all_data)

(15563, 15563, 31126)

In [39]:
from transformers import AutoTokenizer

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-af-fr")

# Your corpus of data (replace this with your actual data)
corpus = all_data

# Function to find tokens mapped to <unk>
def find_unk_tokens(text):
    tokens = tokenizer.tokenize(text)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    unk_tokens = [token for token, id in zip(tokens, ids) if id == 1]
    return unk_tokens

# Process the corpus
all_unk_tokens = set()
for sentence in corpus:
    unk_tokens = find_unk_tokens(sentence)
    all_unk_tokens.update(unk_tokens)

# Print the results
print("Tokens mapped to <unk>:")
for token in all_unk_tokens:
    print(token)

/home/rana/Projects/zindi/transformers/src/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Tokens mapped to <unk>:
ɲ
–
’
»
”
«
ɛɲɛ
ɛɲ
ɲɛɛ
ğ
ɛɛ
—«
υ
—
’«
ɲɛɲɛ
“
Õ
ɲɛ
ŋ
̋
ʻ
Ɔ
ɛ
ɛ”
ễ


In [32]:
with open('data/unmapped_fr_tokens.txt','w') as f:
    for token in all_unk_tokens:
         f.write(token + '\n')

In [21]:
len(all_unk_tokens)

26

In [47]:
# tokenizer.add_tokens(list(all_unk_tokens))
tokenizer.save_pretrained("tokenizer_custom/marian_v2")

('tokenizer_custom/marian_v2/tokenizer_config.json',
 'tokenizer_custom/marian_v2/special_tokens_map.json',
 'tokenizer_custom/marian_v2/vocab.json',
 'tokenizer_custom/marian_v2/source.spm',
 'tokenizer_custom/marian_v2/target.spm',
 'tokenizer_custom/marian_v2/added_tokens.json')

In [48]:
# convert vocab to dictionary
import json
# Input and output file paths
json_file_path = "tokenizer_custom/marian_v2/vocab.json"
text_file_path = "tokenizer_custom/marian_v2/dictionary.txt"

# Read the JSON file
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Write to the text file
with open(text_file_path, 'w') as text_file:
    for token, _ in data.items():
        text_file.write(f"{token} 1\n")

print(f"Conversion completed. Output written to {text_file_path}")

Conversion completed. Output written to tokenizer_custom/marian_v2/dictionary.txt


In [50]:
from unicodedata2 import *
from collections import Counter
from tqdm import tqdm
import sentencepiece as spm
import sentencepiece.sentencepiece_model_pb2 as model


tok_exclusion = ['<s>', '<pad>', '<unk>', '</s>']


newdict2 = []
with open('tokenizer_custom/marian_v2/dictionary.txt', 'r', encoding='utf-8') as f:
    for line in f:
        token = line.strip().split()[0]
        newdict2.append(token)


serializedStr=open('tokenizer_custom/marian_v2/source.spm', 'rb').read()
m=model.ModelProto()
m.ParseFromString(serializedStr)
curdict = []
for i in tqdm(range(len(m.pieces) - 1, 2, -1)):
    curdict.append(m.pieces[i].piece)
    if m.pieces[i].piece not in newdict2:
        hex_string = "".join("{:02x}".format(ord(c)) for c in m.pieces[i].piece)
        print("Removing: ", hex_string, " from spm model, not in dict. Index: ", i)
        m.pieces.pop(i)

for tok in tqdm(newdict2):
    if (tok not in curdict) and (tok not in tok_exclusion):
        print("Adding: ", tok, " to spm model")
        newtoken = m.SentencePiece()
        newtoken.piece = tok
        newtoken.score = 0
        m.pieces.append(newtoken)
        
print(len(m.pieces))
        
with open('tokenizer_custom/marian_v2/source_mod.spm', 'wb') as f:
    f.write(m.SerializeToString())

  0%|          | 0/31997 [00:00<?, ?it/s]

  0%|          | 48/31997 [00:00<02:12, 241.88it/s]

Removing:  1eb8  from spm model, not in dict. Index:  31993


  0%|          | 137/31997 [00:00<01:53, 279.94it/s]

Removing:  25a1  from spm model, not in dict. Index:  31925
Removing:  25cf  from spm model, not in dict. Index:  31924
Removing:  25c6  from spm model, not in dict. Index:  31922


...

 81%|████████  | 47962/59440 [01:48<00:31, 362.85it/s]

Adding:  ▁Canadienne  to spm model
Adding:  ▁Chrétiens  to spm model
Adding:  ▁Concentr  to spm model
Adding:  ▁Contenu  to spm model
Adding:  ▁Créé  to spm model
Adding:  ▁Darby  to spm model
Adding:  ▁Définiss  to spm model
Adding:  ▁Désactive  to spm model
Adding:  ▁Except  to spm model
Adding:  ▁Fille  to spm model
Adding:  ▁GÉNÉRALE  to spm model
Adding:  ▁Hazaël  to spm model
Adding:  ▁Ibid  to spm model
Adding:  ▁Influencé  to spm model
Adding:  ▁Inscription  to spm model
Adding:  ▁Janvier  to spm model
Adding:  ▁Macédonien  to spm model
Adding:  ▁Manque  to spm model
Adding:  ▁Méta  to spm model
Adding:  ▁Navigation  to spm model
Adding:  ▁Nécessit  to spm model
Adding:  ▁Occidentale  to spm model


 81%|████████  | 47992/59440 [01:48<00:25, 443.49it/s]


Adding:  ▁PLU  to spm model
Adding:  ▁PRÊT  to spm model
Adding:  ▁Pench  to spm model
Adding:  ▁Phare  to spm model
Adding:  ▁Photographie  to spm model
Adding:  ▁Pourcentage  to spm model
Adding:  ▁Prière  to spm model
Adding:  ▁Redimensionner  to spm model


KeyboardInterrupt: 